In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class ODE:
    def __init__(self, data):
        for key, value in data.items():
            setattr(self, key, value)

        if not getattr(self, "state_keys",0):
            self.state_keys = list(data.keys())
        if not getattr(self, "timestep",0):
            self.timestep = 1
        for key in self.state_keys:
            setattr(self, key+"0", data[key])

    def __str__(self):
        return str(self.__dict__)

    def get_state(self):
        """Update state vector to values given by input"""
        x = [getattr(self,key) for key in self.state_keys]
        return x

    def update_state(self, x_new):
        """Update state vector to values given by input"""
        for key, val in zip(self.state_keys, x_new):
            setattr(self, key, val)
        return

    def reset(self):
        """Resets state to x0"""
        x0 = [getattr(self,key+"0") for key in self.state_keys]
        self.update_state(x0)
        return

    def time_arr(self, length):
        return np.arange(0, length*self.timestep, self.timestep)

    def euler_step(self, dx):
        """
        Updates state using state vector derivative and one step of eulers method.
        
        Parameters
        ----------
        dx : numpy array
            Derivative of state vector.
        """
        x_new = self.get_state() + dx * self.timestep
        self.update_state(x_new)
        return

pancreas_data = {
    "state_keys" : ["M", "P", "R", "gamma", "D", "DIR", "rho"],
    "M": 779866,
    "P" : 220,
    "R" : 0,
    "gamma" : 0,
    "D" : 0,
    "DIR" : 0,
    "rho" : 0,

    "Gu" : 4.5,
    "Gl" : 9,
    "alpha1" : 50.4934,
    "delta1" : 3.98e-4,
    "v" : 3.16e-5,
    "delta2" : 0.3,
    "k" : 0.1,
    "eta" : 4,
    "gammab" : 1e-5,
    "hhat" : 1.38e-3,
    "zeta" : 4,
    "rhob" : 0.015,
    "fb" : 0.05,
    "k1p" : 5.788e-5,
    "CT" : 300,
    "k1m" : 0.255,
    "krho" : 1050,
    "I0" : 1.6,
    "Kf" : 3.43,
    "N" : 2.76e+6 
}

class PKPM(ODE):
    def __init__(self, **kwargs):
        defaults = pancreas_data
        self.timestep = 5
        super().__init__(defaults)

    def eval(self, G):
        if G <= self.Gu:
            alpha1 = self.alpha1
            alpha2 = 0
            delta1 = self.delta1
            f1 = self.fb
            v = self.v
        else:
            alpha1 = 116.98
            delta1 = 1.5e-4
            v = 7.89e-5
            f1 = self.fb + (1 - self.fb) *  (G - self.Gu) / (self.Kf +  G - self.Gu)
            if G <= self.Gl:
                alpha2 = self.hhat * (G - self.Gu)/(G - self.Gu)
            else:
                alpha2 = self.hhat

        dM = alpha1 - delta1 * self.M
        dP = v * self.M - self.delta2 * self.P - self.k * self.P * self.rho * self.DIR
        dR =  self.k * self.P * self.rho * self.DIR - self.gamma * self.R
        dgamma = self.eta * (-self.gamma + self.gammab + alpha2)
        dD = self.gamma * self.R - self.k1p * (self.CT - self.DIR) * self.D - self.k1m * self.DIR - self.rho * self.DIR
        dDIR = self.k1p * (self.CT - self.DIR) * self.D - self.k1m * self.DIR - self.rho * self.DIR
        drho = self.zeta * (self.rho + self.rhob + self.krho * (self.gamma - self.gammab))
        
        dx = np.array([dM, dP, dR, dgamma, dD, dDIR, drho])
        ISR = self.I0 * self.rho * self.DIR * self.f1 * self.N
        x_new = dx * self.timestep + self.get_state()
        self.update_state(x_new)
        return ISR

    def simulate(self,G_list):
        res = []
        for G in G_list:
            dx = self.PKPM(G)
            self.euler_step(dx)
            res.append(self.get_state())
        return np.array(res)


In [19]:
data = {
    "SRs": 0,
    "alpha": 89.9,
    "gamma": 0.76,
    "KD" : 6.6,
    "h" : 92.5
}
class Pancreas(ODE):
    def __init__(self, model, patient, **kwargs):
        self.timestep = patient.timestep
        self.Gbar = patient.Gbar
        self.model = model.upper()
        with open('config.json', 'r') as f:
            defaults = json.load(f)[self.model]
        defaults.update(kwargs)
        super().__init__(defaults)



class SD(ODE):
    def __init__(self,alpha, gamma, h, KD, ybar, timestep):
        data = {
            "state_keys" : ["SRs", "yprev"],
            "SRs" : 0,
            "ybar" : ybar,
            "yprev" : ybar,
            "alpha" : alpha,
            "gamma" : gamma,
            "h" : h,
            "KD" : KD,
            "timestep" : timestep
        }
        super().__init__(data)

    
    def eval(self,y):
        dy = (y - self.yprev)/self.timestep # approx derivative of y
        dSRs = -self.alpha * (self.SRs + self.gamma * (self.h - y))
        SRD = max(dy * self.KD, 0)
        res = max(self.SRs + SRD,0)

        self.SRs += dSRs * self.timestep
        self.yprev = y
        return res


class PID(ODE):
    def __init__(self, Kp, Td, Ti, ybar, timestep):
        data = {
            "state_keys" : ["I", "yprev"],
            "I" : 0,
            "yprev" : ybar,
            "Kp" : Kp,
            "Td" : Td,
            "Ti" : Ti,
            "ybar" : ybar,
            "timestep" : timestep
        }
        super().__init__(data)

    def eval(self,y):
        dy = (y - self.yprev)/self.timestep # approx derivative of y
        ek = y - self.ybar # error

        P = self.Kp * ek
        dI = P/self.Ti
        D = self.Kp * self.Td * dy

        res = max(P + self.I + D, 0)

        self.yprev = y 
        self.I += dI * self.timestep # Updates integral term
        return res        

pan = SD(3,3,3,3,3,3)
pan.eval(4)
print(pan)

{'state_keys': ['SRs', 'yprev'], 'SRs': 27, 'ybar': 3, 'yprev': 4, 'alpha': 3, 'gamma': 3, 'h': 3, 'KD': 3, 'timestep': 3, 'SRs0': 0, 'yprev0': 3}


In [ ]:
p = patient()
p.pancreas.get_insulin()